BYNmasivofy2024*

In [365]:
# Importa las librerias a usar
import pandas as pd
import re
import numpy as np


importar y procesar el K2

In [366]:

# Ubicacion de los archivos a usar
patch1 = '../input/Kilometros en Servicio MIO - 05-Mar-2024.xlsx'# Se importa el archivo xlxs
kmsmio = pd.read_excel(patch1, sheet_name='Programacion')
# Se eliminan las filas y las columnas vacias
kmsmio = kmsmio.dropna(axis=0, how='all').dropna(axis=1, how='all')


In [367]:
# Filtra el DF por Kms en Servicio
kmsmio = kmsmio[kmsmio['Tipo Kilómetros']== 'Kms en Servicio']
kmsmio = kmsmio[kmsmio['Kilometros Programados Plan'].notna()]

In [368]:
# Cambiamos la hora 24 por 0
kmsmio['Desde'] = kmsmio['Desde'].replace('24', '0')
kmsmio = kmsmio[kmsmio['Desde'] != '0']


In [369]:
# Se crea una nueva columna 'id'
kmsmio['id'] = kmsmio.apply(lambda row: '-'.join([str(row['Línea']), str(row['Número de tarea vehículo']), 
                                                   str(row['Desde']), str(row['hasta'])]), axis=1)


In [370]:
# Se verifica el tamaño del df
kmsmio.shape

(18645, 40)

In [371]:
# Se verifica que la columna id no tenga id repetidos
kmsmio['id'].value_counts().sort_values(ascending=False)

id
A14A-1-05:00:00-05:12:10                              1
T47B-2-19:21:48-20:01:10                              1
A14A-1-05:26:00-05:38:10                              1
A14A-1-05:38:55-05:51:15                              1
A14A-1-05:52:00-06:04:43                              1
                                                     ..
A35A-99753-1900-01-01 00:03:04-1900-01-01 00:14:39    1
A35A-99753-23:03:04-23:14:39                          1
A35A-99753-21:33:04-21:44:07                          1
A35A-99753-21:30:02-21:30:17                          1
A35A-99753-1900-01-01 00:16:10-1900-01-01 00:29:23    1
Name: count, Length: 18645, dtype: int64

Importar reporte finalizado

In [372]:
# ubicacion del archivo
patch2 =  '../input/REPORTE FINALIZADO 05_MARZ_2024.xlsx'
# Se importa el archivo xlxs
repofi = pd.read_excel(patch2, sheet_name='Programacion')
bitacora = pd.read_excel(patch2, sheet_name='Bitacora')
# Se eliminan las filas y las columnas vacias
repofi = repofi.dropna(axis=0, how='all').dropna(axis=1, how='all')
bitacora = bitacora.dropna(axis=0, how='all').dropna(axis=1, how='all')


In [373]:
# Filtra el DF por Kms en Servicio
repofi = repofi[repofi['Tipo de kilometros']== 'Kms en Servicio']
# Se crea una nueva columna 'id'
repofi['id'] = repofi.apply(lambda row: '-'.join([str(row['Línea']), str(row['Número de tarea vehículo']), 
                                                  str(row['desde']),str(row['hasta'])]), axis=1)


In [374]:
# Se verifica que el id no tenga datos repetidos
repofi['id'].value_counts().sort_values(ascending=False)

id
A14A-1-05:00:00-05:12:10                              1
T47B-2-19:21:48-20:01:10                              1
A14A-1-05:26:00-05:38:10                              1
A14A-1-05:38:55-05:51:15                              1
A14A-1-05:52:00-06:04:43                              1
                                                     ..
A35A-99753-1900-01-01 00:03:04-1900-01-01 00:14:39    1
A35A-99753-23:03:04-23:14:39                          1
A35A-99753-21:33:04-21:44:07                          1
A35A-99753-21:30:02-21:30:17                          1
A35A-99753-1900-01-01 00:16:10-1900-01-01 00:29:23    1
Name: count, Length: 18645, dtype: int64

In [375]:
# Se verifica el tamaño del df
repofi.shape

(18645, 32)

Se realiza el respectivo Merge con los dos df

In [376]:
# Se realiza el merge con outer en la columna id
merge = pd.merge(kmsmio, repofi, on='id', how='outer')

In [377]:
# Se verifica que el numero de filas en los df sea igual (True), para poder concluir que el merge fue exitoso
merge.shape[0] == kmsmio.shape[0] == repofi.shape[0]

True

In [378]:
merge.shape

(18645, 71)

In [379]:
# Exporta a un archivo excel el Df
#merge.to_excel('../output/prueba_merge.xlsx', index=False, sheet_name='k2_procesado')

Merge con bitacora


In [380]:
merge[['Número de tarea vehículo_x', 'Número de viaje_x']]

,Número de tarea vehículo_x,Número de viaje_x
0,13,1
1,13,2
2,13,3
3,13,4
4,13,5
...,...,...
18640,85,175
18641,85,9983
18642,85,194
18643,85,9982


In [381]:
bitacora[['Tarea', 'Viaje']] = bitacora[['Tarea', 'Viaje']].fillna(0).astype(int)

In [382]:
# Se selecionan las columnas de Bitacora que se van a usar
bitacorafil = bitacora[['Jerarquía',	'Tarea',	'Viaje', 'Parada', 'Vehículo', 'Novedad']]
# Se filtran los casos que no tiene numero de viaje ya que no tiene retoma
bitacorafil = bitacorafil[bitacorafil['Viaje'] != 0]

In [383]:
# Se compara las filas de los df, el resultado debe ser False
bitacora.shape[0]==bitacorafil.shape[0]

False

In [384]:
# Se agrega al merge anterior la informacion de bitacora
# Importante tener en cuenta los vehiculos que tengan mas de un caso en la bitacora, esto genera id duplicados.
mergebitacora = pd.merge(merge, bitacorafil, left_on=['Número de tarea vehículo_x', 'Número de viaje_x'], right_on=['Tarea', 'Viaje'], how='outer' )

In [385]:
# Revisar este resultado, ya que los valore sque no coinceden deven se iguales en el filtro
mergebitacora.shape[0]- merge.shape[0]

34

In [386]:
mergebitacora[mergebitacora['id'].duplicated()]

,Fecha_x,Día tipo_x,Designación de tarea vehículo_x,Línea_x,Tipo de viaje corto_x,Sentido_x,Tipo Kilómetros,Número de Vehículo,Concesionario de Transporte_x,Número de tarea vehículo_x,...,Vehículo_x,Kms Planeados,Kms Perdidos,Kms cancelados,Jerarquía,Tarea,Viaje,Parada,Vehículo_y,Novedad
326,2024-03-05,DHABIL,13,A01A,Viaje comercial,Ida,Kms en Servicio,54013,BYNCPC,13,...,54013.0,4704,NaN,NaN,Salida de Operación::Por Flota::Motor,13.0,21.0,SANBO2,54013.0,Motor
332,2024-03-05,DHABIL,13,A01A,Viaje comercial,Ida,Kms en Servicio,54013,BYNCPC,13,...,54013.0,4704,4166.0,NaN,Salida de Operación::Por Flota::Motor,13.0,26.0,SANPAS6,54013.0,Motor
334,2024-03-05,DHABIL,13,A01A,Viaje comercial,Ida,Kms en Servicio,NaN,NaN,13,...,22117.0,4704,4704.0,NaN,Retoma,13.0,27.0,SANBO2,22117.0,Retoma
379,2024-03-05,DHABIL,14,A32,Viaje comercial,Ida,Kms en Servicio,NaN,NaN,14,...,23033.0,4430,NaN,NaN,Retoma,14.0,48.0,C62K1BIS,23033.0,Retoma
1158,2024-03-05,DHABIL,42,P21A,Viaje comercial,Ida,Kms en Servicio,22142,BYN,42,...,22142.0,14534,NaN,NaN,Salida de Operación::UTRYT::IBIS,42.0,109.0,UNIVCOM1,22142.0,IBIS
1689,2024-03-05,DHABIL,61,A44B,Viaje comercial,Ida,Kms en Servicio,23051,BYN,61,...,23051.0,7651,NaN,NaN,Salida de Operación::UTRYT::IBIS,61.0,3.0,TAGUAC7,23051.0,IBIS
2991,2024-03-05,DHABIL,106,E21,Viaje comercial,Ida,Kms en Servicio,NaN,NaN,106,...,21053.0,16401,NaN,NaN,Salida de Operación::Por Flota::Motor,106.0,233.0,UNIV1,21053.0,Motor
4232,2024-03-05,DHABIL,154,A17D,Viaje comercial,Ida,Kms en Servicio,23041,BYN,154,...,NaN,8840,NaN,NaN,Salida de Operación::Por Operador::Abandono De...,154.0,102.0,UNIVCOM5,23041.0,Injustificado
4852,2024-03-05,DHABIL,176,A44A,Viaje comercial,Ida,Kms en Servicio,22029,BYN,176,...,22029.0,6112,518.0,NaN,Incidente::Despacho::Asignacion Incorrecta De ...,176.0,153.0,K29C80,22029.0,Asignacion Incorrecta De Tarea En Ivu Vehicle
4853,2024-03-05,DHABIL,176,A44A,Viaje comercial,Ida,Kms en Servicio,22029,BYN,176,...,22029.0,6112,518.0,NaN,Salida de Operación::Por Operador::Abandono De...,176.0,153.0,TAGUAC7,22029.0,NO RELEVO


Columna condicional 

In [387]:
# Se verifica si falta el numero de vehiculo en el reporte de cierre para asumir el de metrocali
#mergebitacora['Número de secuencia de carro'] = [x if pd.notna(x) else y for x, y in zip(mergebitacora['Número de secuencia de carro'], mergebitacora['Número de Vehículo'])]


In [388]:
mergebitacora[['Largo_x','Largo_y', 'Número de Vehículo','Número de secuencia de carro' ]]

,Largo_x,Largo_y,Número de Vehículo,Número de secuencia de carro
0,5311,5311,32013,NaN
1,4419,4419,32013,NaN
2,5337,5311,32013,NaN
3,4419,4419,32013,NaN
4,5337,5311,32013,NaN
...,...,...,...,...
18674,77,77,53009,53009
18675,77,77,53009,53009
18676,77,77,53009,53009
18677,77,77,53009,53009


In [389]:
# Se cambian los vehiculos del cierre por los de k2
mergebitacora['Número de secuencia de carro'] = [x if pd.notna(x) else y for x, y in zip(mergebitacora['Número de secuencia de carro'], mergebitacora['Número de Vehículo'])]
mergebitacora['cambios'] = np.where(mergebitacora['Número de secuencia de carro'] != mergebitacora['Número de Vehículo'], 'Se cambia numero de vehiculo', '# V. Sin Cambio')


In [391]:
# Función para actualizar los valores de Largo_y si Largo_x es mayor
def actualizar_largo(row):
    if pd.isna(row['Largo_y']):
        return row['Largo_x']
    elif row['Largo_x'] > row['Largo_y']:
        return row['Largo_x']
    else:
        return row['Largo_y']

In [392]:
# Aplicar la función a la columna Largo_y
mergebitacora['Largo_y'] = mergebitacora.apply(actualizar_largo, axis=1)

# Marcar las filas donde se realizan cambios en una nueva columna
mergebitacora['cambios'] = mergebitacora.apply(lambda row: 'Se cambia Largo' if pd.isna(row['cambios']) else row['cambios'] + '/Se cambia Largo_y' 
                                               if ('Se cambia Largo_y' not in row['cambios']) and (row['Largo_x'] != row['Largo_y']) else row['cambios'] 
                                               if (row['Largo_x'] == row['Largo_y']) and ('Se cambia Largo_y' in row['cambios']) else 'Largo Sin Cambio', axis=1)


In [393]:
mergebitacora['cambios'].value_counts()

cambios
Largo Sin Cambio                                  18189
# V. Sin Cambio/Se cambia Largo_y                   479
Se cambia numero de vehiculo/Se cambia Largo_y       11
Name: count, dtype: int64

In [322]:
def actualizar_secuencia_carro(row):
    if pd.isna(row['Número de secuencia de carro']):
        if pd.notna(row['Número de Vehículo']) or (pd.notna(row['Largo_x']) and pd.notna(row['Largo_y'])):
            if row['Largo_x'] > row['Largo_y']:
                return row['Número de Vehículo']
            else:
                return np.nan
    return row['Número de secuencia de carro']


def condicion_cambio(row):
    if pd.isna(row['Número de secuencia de carro']):
        if pd.notna(row['Número de Vehículo']):
            if row['Largo_x'] > row['Largo_y']:
                return 'Cambio realizado: largo_x y número de secuencia de carro'
            else:
                return 'Cambio realizado: número de secuencia de carro'
    return np.nan


mergebitacora['Número de secuencia de carro'] = mergebitacora.apply(actualizar_secuencia_carro, axis=1)
mergebitacora['condicion'] = mergebitacora.apply(condicion_cambio, axis=1)



In [323]:
mergebitacora['condicion'].value_counts()

condicion
Cambio realizado: número de secuencia de carro    9286
Name: count, dtype: int64

In [332]:
# Condiciones para la nueva columna
condicion1 = mergebitacora['Largo_x'] != mergebitacora['Largo_y']
condicion2 = mergebitacora['Número de Vehículo'] != mergebitacora['Número de secuencia de carro']
condicion3 = mergebitacora['Secuencia de arcos_x'] != mergebitacora['Secuencia de arcos_y']
condicion4 = mergebitacora['Concesionario de Transporte_x'] != mergebitacora['Concesionario Programado']

# Aplicar condiciones y asignar valores a la nueva columna
mergebitacora['condicional'] = np.where(condicion1 & condicion2 & condicion3, 'Revisar largo, parada y vehículo',
                                 np.where(condicion1, 'Revisar largo', 
                                          np.where(condicion2, 'Revisar numero de vehiculo',
                                                   np.where(condicion3, 'Revisar paradas',
                                                            np.where(condicion4, 'Revisar consesionario', 'OK')))))

In [333]:
mergebitacora['condicional'].value_counts()

condicional
Revisar consesionario         10345
OK                             4804
Revisar numero de vehiculo     2657
Revisar largo                   873
Name: count, dtype: int64